In [1]:
from scipy import stats
from data_processing import get_paths_folders

import pandas as pd
import numpy as np

In [2]:
path = "..\\results"

In [3]:
def slice_dataframe(data):
    UniqueNames = data.organism.unique()

    #create a data frame dictionary to store your data frames
    DataFrameDict = {elem : pd.DataFrame for elem in UniqueNames}
    for key in DataFrameDict.keys():
        DataFrameDict[key] = data[data.organism == key][:].reset_index().drop('index', axis=1)
        
    return DataFrameDict

In [9]:
d = get_paths_folders(path, False)
results_classifiers = {}
for dirname in d: #\results\j48 (if not threshold)   
    if dirname.__contains__("threshold"):
        results_classifiers[dirname] = {}
        newPath = path + "\\" + dirname      
        d1 = get_paths_folders(newPath, False)
        for dirname1 in d1:
            results_classifiers[dirname][dirname1] = {}
            newPath2 = newPath + "\\" + dirname1
            f = get_paths_folders(newPath2, True)
            for filename in f:        
                df = pd.read_csv(newPath2 + "\\" + filename)
                df.drop(df.filter(regex="Unnamed"), axis=1, inplace=True)   
                df = df.replace(",", ".", regex=True)
                df = df.astype({"GMean": float, "ratioReduction": float})
                DataFrameDict = slice_dataframe(df)
                for key in DataFrameDict.keys():           
                    k = 0                   
                    for value in DataFrameDict[key]["GMean"]: 
                        if value <= 0:                           
                            value = value * (-1)
                        DataFrameDict[key].at[k, "GMean"] = value
                        k = k + 1                 
                    DataFrameDict[key] =  DataFrameDict[key].sort_values(by='Fold', ascending=True)

                
                for key in DataFrameDict.keys():
                    if key not in results_classifiers[dirname][dirname1].keys():
                        results_classifiers[dirname][dirname1][key] = list()                   
                    results_classifiers[dirname][dirname1][key].append(DataFrameDict[key])

    else:
        results_classifiers[dirname] = {}
        newPath = path + "\\" + dirname
        f = get_paths_folders(newPath, True)
        for filename in f:        
            df = pd.read_csv(newPath + "\\" + filename)
            df.drop(df.filter(regex="Unnamed"), axis=1, inplace=True)   
            df = df.replace(",", ".", regex=True)
            df = df.astype({"GMean": float, "ratioReduction": float})
            DataFrameDict = slice_dataframe(df)
            for key in DataFrameDict.keys():           
                k = 0
                for value in DataFrameDict[key]["GMean"]:             
                    if value <= 0:
                        value = value * (-1)
                    DataFrameDict[key].at[k, "GMean"] = value
                    k = k + 1            
                DataFrameDict[key] =  DataFrameDict[key].sort_values(by='Fold', ascending=True)

            
            for key in DataFrameDict.keys():
                if key not in results_classifiers[dirname].keys():
                    results_classifiers[dirname][key] = list()                   
                results_classifiers[dirname][key].append(DataFrameDict[key])  


In [16]:
print(results_classifiers['KNN']['yeast'][1]['GMean'])

0     0.00
3    49.39
1    55.95
4    48.84
2    37.12
5     0.00
7    57.08
8     0.00
6    56.71
9     0.00
Name: GMean, dtype: float64


In [6]:
stats.friedmanchisquare(results_classifiers['threshold-10']['KNN']['worm'][1]['GMean'],
                        results_classifiers['threshold-10']['KNN']['worm'][0]['GMean'],
                        results_classifiers['threshold-10']['NB']['worm'][1]['GMean'],
                        results_classifiers['threshold-10']['NB']['worm'][0]['GMean'],
                        results_classifiers['threshold-10']['J48']['worm'][1]['GMean'],
                        results_classifiers['threshold-10']['J48']['worm'][0]['GMean']
                        )

FriedmanchisquareResult(statistic=25.8187134502924, pvalue=9.675756054634503e-05)

In [15]:
np.std(results_classifiers['NB']['fly'][1]['GMean'])

17.54998304842486